In [32]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from torchaudio.models.wav2vec2.components import LayerNorm
from torchinfo import summary
from torchvision import datasets, transforms, models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


## Implementatation

In [33]:
from torchvision.ops.misc import Permute

class ChannelLayerNoorm(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.permute1 = Permute([0, 2, 3, 1])
        self.layer_norm = nn.LayerNorm(channels)
        self.permute2 = Permute([0, 3, 1, 2])
        
    def forward(self, x):
        x = self.permute1(x)
        x = self.layer_norm(x)
        x = self.permute2(x)
        return x


class ConvNeXtBlock(nn.Module):
    def __init__(self, input_channels, inner_channels):
        super().__init__()
        self.residual_block = nn.Sequential(
            nn.Conv2d(input_channels, inner_channels, kernel_size=7, stride=1, padding=3, groups=inner_channels),
            nn.GELU(),
            Permute([0, 2, 3, 1]),
            nn.LayerNorm(input_channels),
            nn.Linear(inner_channels, inner_channels*4),
            nn.GELU(),
            nn.Linear(inner_channels*4, inner_channels),
            Permute([0, 3, 1, 2])
        )
        self.layer_scale = nn.Parameter(torch.full((1, inner_channels, 1, 1), 1e-6))
    
    def forward(self, x):
        shortcut = x
        x = self.residual_block(x) * self.layer_scale
        return x + shortcut


class ConvNeXt(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(n_channels, 96, kernel_size=4, stride=4),
            ChannelLayerNoorm(96),
            nn.GELU(),
        )
        self.layers1 = self.make_layers(96, 96, 3, False)
        self.layers2 = self.make_layers(96, 192, 3, True)
        self.layers3 = self.make_layers(192, 384, 9, True)
        self.layers4 = self.make_layers(384, 768, 3, True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            ChannelLayerNoorm(768),
            nn.Flatten(),
            nn.Linear(768, n_classes)
        )
    
    def make_layers(self, input_channels, inner_channels, n_blocks, downsample):
        layers = [nn.Sequential(
            ChannelLayerNoorm(input_channels),
            nn.Conv2d(input_channels, inner_channels, kernel_size=2, stride=2),
        )] if downsample else []
        for layer in range(n_blocks):
            layers.append(ConvNeXtBlock(inner_channels, inner_channels))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.layers1(x)
        x = self.layers2(x)
        x = self.layers3(x)
        x = self.layers4(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

In [34]:
convnext_model = ConvNeXt(3, 1000)
summary(convnext_model, input_size=(1, 3, 224, 224), col_names=['output_size', 'num_params', 'mult_adds'], depth=4, device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #                   Mult-Adds
ConvNeXt                                 [1, 1000]                 --                        --
├─Sequential: 1-1                        [1, 96, 56, 56]           --                        --
│    └─Conv2d: 2-1                       [1, 96, 56, 56]           4,704                     14,751,744
│    └─ChannelLayerNoorm: 2-2            [1, 96, 56, 56]           --                        --
│    │    └─Permute: 3-1                 [1, 56, 56, 96]           --                        --
│    │    └─LayerNorm: 3-2               [1, 56, 56, 96]           192                       192
│    │    └─Permute: 3-3                 [1, 96, 56, 56]           --                        --
│    └─GELU: 2-3                         [1, 96, 56, 56]           --                        --
├─Sequential: 1-2                        [1, 96, 56, 56]           --                        --
│    └─ConvNeXtBlock: 2-

In [35]:
convnext_torch_model = models.convnext_tiny()
summary(convnext_torch_model, input_size=(1, 3, 224, 224), col_names=['output_size', 'num_params', 'mult_adds'], depth=5, device='cpu')

Layer (type:depth-idx)                        Output Shape              Param #                   Mult-Adds
ConvNeXt                                      [1, 1000]                 --                        --
├─Sequential: 1-1                             [1, 768, 7, 7]            --                        --
│    └─Conv2dNormActivation: 2-1              [1, 96, 56, 56]           --                        --
│    │    └─Conv2d: 3-1                       [1, 96, 56, 56]           4,704                     14,751,744
│    │    └─LayerNorm2d: 3-2                  [1, 96, 56, 56]           192                       192
│    └─Sequential: 2-2                        [1, 96, 56, 56]           --                        --
│    │    └─CNBlock: 3-3                      [1, 96, 56, 56]           96                        --
│    │    │    └─Sequential: 4-1              [1, 96, 56, 56]           --                        --
│    │    │    │    └─Conv2d: 5-1             [1, 96, 56, 56]           4,8

## Training

In [36]:
from pathlib import Path

TRAIN_RATIO = 0.8
data_dir = Path('./data/')

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

train_ds = datasets.CIFAR100(data_dir, train=True, download=True, transform=transform)
train_ds, val_ds = random_split(train_ds, (TRAIN_RATIO, 1 - TRAIN_RATIO))
val_ds.transform = transform
test_ds = datasets.CIFAR100(data_dir, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [37]:
import wandb
from src.engine import *

config = dict(batch_size=64, lr=5e-4, epochs=20, dataset='CIFAR100')
with wandb.init(project='pytorch-study', name='ConvNeXt50', config=config) as run:
    w_config = run.config
    train_dl = DataLoader(train_ds, batch_size=w_config.batch_size, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=w_config.batch_size, shuffle=True)
    
    n_classes = len(train_ds.dataset.classes)
    convnext_mdoel = ConvNeXt(3, n_classes).to(DEVICE)
        
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(convnext_mdoel.parameters(), lr=w_config.lr)
    
    loss_history, acc_history = train(convnext_mdoel, train_dl, val_dl, criterion, optimizer, w_config.epochs, DEVICE, run) 

Epoch=20: 100%|██████████| 20/20 [2:44:53<00:00, 494.67s/it, train_loss=0.112, train_acc=96.36%, val_loss=4.290, val_acc=44.60%]  
